### Imports

In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from datetime import datetime
from bokeh.palettes import Spectral3
from tabulate import tabulate
from bokeh.transform import factor_cmap
import js2py
import math
from concurrent.futures import ProcessPoolExecutor as PoolExecutor
import requests
import os

# convenience
output_notebook()
pd.set_option('display.max_colwidth', -1)

Loading BokehJS ...

### Globals

In [2]:
API = os.getenv('API')
NUM_CHALLENGES = 100
NUM_BLOCKS = 100
TOP = 10
animalhash = js2py.require('angry-purple-tiger')
MAX_WORKERS = 8
REWARD_LIMIT = 500

In [5]:
API

'https://testing.helium.foundation'

### Functions

In [6]:
def latest_height(api):
    '''
    Get latest block height
    '''
    r = requests.get(f'{api}/blocks?limit=1')
    if r.status_code == 200:
        data = r.json().get('data')
        return data[0]['height']
    raise Exception('Unable to get latest height')

def get_blocks(api, num_blocks=100):
    '''
    Get past `NUM_BLOCKS` number of num_blocks
    '''
    r = requests.get(f'{api}/blocks?limit={num_blocks}')
    if r.status_code == 200:
        data = r.json().get('data')
        assert len(data) == num_blocks
        return data
    raise Exception('Unable to get blocks')

def get_challenges(api, num_challenges=100):
    '''
    Get past `LIMIT` number of challenges
    '''
    r = requests.get(f'{api}/challenges?limit={num_challenges}')
    if r.status_code == 200:
        data = r.json().get('data')
        assert len(data) == num_challenges
        return accumulate_results(data)
    raise Exception('Unable to get challenges')

def get_hotspots(api):
    '''
    Get hotspots from the api
    '''
    r = requests.get(f'{api}/hotspots')
    if r.status_code == 200:
        data = r.json().get('data')
        return data
    raise Exception('Unable to get hotspots')

def get_accounts(api):
    '''
    Get accounts from the api
    '''
    r = requests.get(f'{api}/accounts')
    if r.status_code == 200:
        data = r.json().get('data')
        return data
    raise Exception('Unable to get accounts')

def accumulate_results(challenges):
    '''
    Return a dict list of challenge_id, num_successes, num_failures and num_untested
    '''
    results = []
    for challenge in challenges:
        path = challenge['pathElements']

        result = [element['result'] for element in path]
        length = len(result)
        num_untested = result.count('untested')

        gray = True if num_untested == length else False

        if gray:
            num_successes = 0
            num_failures = 0
            num_untested = 0
            dud = 1
            success = 0
        else:
            num_successes = result.count('success')
            num_failures = result.count('failure')
            num_untested = num_untested
            dud = 0
            success = 1 if num_successes == length else 0
        results.append({'id': challenge['id'],
                        'num_successes': num_successes,
                        'num_failures': num_failures,
                        'num_untested': num_untested,
                        'dud': dud,
                        'success': success})
    return results


def get_reward(reward_url):
    '''
    Get hotspot reward
    '''
    r = requests.get(reward_url)
    if r.status_code == 200:
        data = r.json().get('data')
        return data
    raise Exception(f'Unable to get rewards for {reward_url}')
    
def get_rewards(api, hotspots, limit=REWARD_LIMIT):
    '''
    Get rewards for all hotspots
    '''
    reward_urls = [f'{api}/hotspots/{hotspot}/rewards?limit={limit}' for hotspot in hotspots]
    with PoolExecutor(max_workers=MAX_WORKERS) as executor:
        rewards = []
        for hotspot, i in zip(hotspots, executor.map(get_reward, reward_urls)):
            rewards.extend(i)
    return rewards

### Block Stats

In [7]:
blocks = get_blocks(API, NUM_BLOCKS)

Exception: Unable to get blocks

In [ ]:
df = pd.DataFrame(blocks)

In [ ]:
print(f'Average txns past {NUM_BLOCKS} blocks: {df["txns"].mean()}')

In [ ]:
print(f'Average block time past {NUM_BLOCKS} blocks: {df["time"].mean()}')

In [ ]:
max_txns = tabulate(df.max().reset_index(),tablefmt='psql', showindex='never')
print(f"Block with max txns for past {NUM_BLOCKS} blocks: \n{max_txns}")

In [ ]:
source = ColumnDataSource(df)
p = figure()

p.vbar(x='height', top='txns', width=0.8, source=source)
hover = HoverTool(tooltips=[('height', '@height'), ('txns', '@txns')])
p.add_tools(hover)
p.xaxis.axis_label = "Txns/Block"
p.yaxis.axis_label = "NumTxns"

show(p)

### Hotspot Stats

In [ ]:
hotspots = get_hotspots(API)

In [ ]:
hdf = pd.DataFrame(hotspots)
hdf['name'] = hdf.apply(lambda row: '-'.join(animalhash(row.address).lower().split()), axis=1)

In [ ]:
print(f'Top {TOP} scoring hotspots:')
hdf.sort_values('score', ascending=False)[['name', 'score']].head(TOP)

In [ ]:
source = ColumnDataSource(hdf)
# p = figure()
p = figure(x_range=hdf['name'], y_range=[0,1], title="Hotspot Scores")

p.vbar(x='name', top='score', width=0.8, source=source)
hover = HoverTool(tooltips=[('name', '@name'), ('score', '@score')])
p.width = 900
p.add_tools(hover)
p.xaxis.axis_label = "Name"
p.yaxis.axis_label = "Score"
p.xaxis.major_label_text_font_size = "4pt"
p.xaxis.major_label_orientation = math.pi/4

show(p)

### Account stats

In [ ]:
accounts = get_accounts(API)

In [ ]:
df = pd.DataFrame(accounts)

In [ ]:
print(f'Top {TOP} account balances:')
df.sort_values('balance', ascending=False)[['address', 'balance']].head(TOP)

### Challenge Stats

In [ ]:
challenges = get_challenges(API, NUM_CHALLENGES)

In [ ]:
df = pd.DataFrame(challenges)

In [ ]:
print(f"Total Successful challenges past {NUM_CHALLENGES} challenges: {sum(df['success'] == 1)}")

In [ ]:
print(f"Entirely dud challenges since past {NUM_CHALLENGES} challenges: {sum(df['dud'] == 1)}")

### Hotspot Reward Stats

In [ ]:
rewards = get_rewards(API, hdf['address'].values)

In [ ]:
rdf = pd.DataFrame(rewards)[['gateway', 'amount', 'account']]

In [ ]:
g1 = rdf.groupby(['gateway', 'account']).sum().reset_index()
g1['name'] = g1.apply(lambda row: '-'.join(animalhash(row.gateway).lower().split()), axis=1)
g1['amount'] = g1.apply(lambda row: row.amount/100000000, axis=1)

In [ ]:
print(f'Top {TOP} rewarded hotspots:')
g1.sort_values('amount', ascending=False)[['name', 'amount', 'account']].head(TOP)

In [ ]:
source = ColumnDataSource(g1)
p = figure(x_range=g1['name'], title=f"Aggregated last {REWARD_LIMIT} hotspot rewards")

p.vbar(x='name', top='amount', width=0.8, source=source)
hover = HoverTool(tooltips=[('name', '@name'), ('amount', '@amount')])
p.width = 900
p.add_tools(hover)
p.xaxis.axis_label = "Hotspot"
p.yaxis.axis_label = "AggregateRewards"
p.xaxis.major_label_text_font_size = "4pt"
p.xaxis.major_label_orientation = math.pi/4

show(p)